# The Helmert transform
$
\begin{bmatrix}
    x\\
    y\\
    z\\
\end{bmatrix}^B
=
\begin{bmatrix}
    t_X\\
    t_Y\\
    t_Z\\
\end{bmatrix}
+
\begin{bmatrix}
    1+S  & -r_Z & r_Y\\
    r_Z  & 1+s  & -r_X\\
    -r_Y & r_X  & 1+S\\
\end{bmatrix}
\cdot
\begin{bmatrix}
    x\\
    y\\
    z\\
\end{bmatrix}^A
$

In [17]:
def helmert(x, y, z=0):
    """ Example implementation of Helmert transform """
    tX = -446.448  
    tY = 125.157  
    tZ = -542.060  

    rX = -0.1502  
    rY = -0.2470  
    rZ = -0.8421  

    s = 20.4894 * math.pow(10, -6)
    # For multiple x and y
    # A_vector = np.matrix([[x, y, z], [x, y, z]]).T
    A_vector = np.vstack(np.array([x, y, z]))
    t_matrix = np.vstack(np.array([tX, tY, tZ]))
    conversion = np.matrix([[1 + s, -rZ, rY], [rZ, 1 + s, -rX], [-rY, rX, 1 + s]])
    return t_matrix + (conversion * A_vector)

In [18]:
helmert(-2.018304, 54.589097)

matrix([[-402.49686677, -402.49686677],
        [ 181.4468293 ,  181.4468293 ],
        [-550.75780346, -550.75780346]])

### Using an external Rust library to speed up lon, lat to [BNG](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) conversion

In [2]:
import numpy as np
import pandas as pd
import math
from ctypes import cdll, c_float, c_double, Structure, ARRAY, POINTER, c_int32, c_uint32, c_size_t, c_void_p, cast
from sys import platform
from bng import bng
import pyproj
import ipdb
from array import array

### Setting up the Rust library. See [here](https://github.com/alexcrichton/rust-ffi-examples/tree/master/python-to-rust) for more

Ensure you've built your Rust library using `cargo build --release`, or the next step will fail.

The boilerplate below can easily be hidden in a wrapper function – it's just here to demonstrate how to call into a shared Rust lib using FFI.

In [3]:
if platform == "darwin":
    ext = "dylib"
else:
    ext = "so"
    
lib = cdll.LoadLibrary('target/release/liblonlat_bng.' + ext)

Define the `ctypes` structures for lon, lat --> BNG conversion

In [4]:
class BNG_FFITuple(Structure):
    _fields_ = [("a", c_uint32),
                ("b", c_uint32)]

class BNG_FFIArray(Structure):
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    # Allow implicit conversions from a sequence of 32-bit unsigned
    # integers.
    @classmethod
    def from_param(cls, seq):
        return seq if isinstance(seq, cls) else cls(seq)

    # Wrap sequence of values. You can specify another type besides a
    # float.
    def __init__(self, seq, data_type = c_float):
        try:
            len(seq)
        except TypeError:
             # we've got an iterator or a generator, so consume it
            seq = array('f', seq)
        array_type = data_type * len(seq)
        try:
            raw_seq = array_type.from_buffer(seq.astype(np.float32))
        except (TypeError, AttributeError):
            try:
                raw_seq = array_type.from_buffer_copy(seq.astype(np.float32))
            except (TypeError, AttributeError):
                # it's a list or a tuple
                raw_seq = array_type.from_buffer(array('f', seq))
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)

class BNG_RESTuple(Structure):
    _fields_ = [("e", BNG_FFIArray),
                ("n", BNG_FFIArray)]        

# A conversion function that cleans up the result value to make it
# nicer to consume.
def bng_void_array_to_tuple_list(restuple, _func, _args):
    eastings = POINTER(c_int32 * restuple.e.len).from_buffer_copy(restuple.e)[0]
    northings = POINTER(c_int32 * restuple.n.len).from_buffer_copy(restuple.n)[0]
    res_list = [list(eastings), list(northings)]
    drop_bng_array(restuple.e, restuple.n)
    return res_list

Define the `ctypes` structures for BNG --> lon, lat conversion

In [5]:
class LONLAT_FFITuple(Structure):
    _fields_ = [("a", c_float),
                ("b", c_float)]

class LONLAT_FFIArray(Structure):
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    # Allow implicit conversions from a sequence of 32-bit unsigned
    # integers.
    @classmethod
    def from_param(cls, seq):
        return seq if isinstance(seq, cls) else cls(seq)

    # Wrap sequence of values. You can specify another type besides a
    # 32-bit unsigned integer.
    def __init__(self, seq, data_type = c_uint32):
        try:
            len(seq)
        except TypeError:
             # we've got an iterator or a generator, so consume it
            seq = array('f', seq)
        array_type = data_type * len(seq)
        try:
            raw_seq = array_type.from_buffer(seq.astype(np.uint32))
        except (TypeError, AttributeError):
            try:
                raw_seq = array_type.from_buffer_copy(seq.astype(np.uint32))
            except (TypeError, AttributeError):
                # it's a list or a tuple
                raw_seq = array_type.from_buffer(array('i', seq))
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)

class LONLAT_RESTuple(Structure):
    _fields_ = [("lon", LONLAT_FFIArray),
                ("lat", LONLAT_FFIArray)]           

# A conversion function that cleans up the result value to make it
# nicer to consume.
def lonlat_void_array_to_tuple_list(array, _func, _args):
    lons = POINTER(c_float * restuple.lon.len).from_buffer_copy(restuple.lon)[0]
    lats = POINTER(c_float * restuple.lat.len).from_buffer_copy(restuple.lat)[0]
    res_list = [list(lons), list(lats)]
    drop_bng_array(restuple.lon, restuple.lat)
    return res_list

Define `ctypes` input and return parameters

In [6]:
# Multi-threaded
convert_bng = lib.convert_to_bng_threaded
convert_bng.argtypes = (BNG_FFIArray, BNG_FFIArray)
convert_bng.restype = BNG_RESTuple
convert_bng.errcheck = bng_void_array_to_tuple_list

convert_lonlat = lib.convert_to_lonlat_threaded
convert_lonlat.argtypes = (LONLAT_FFIArray, LONLAT_FFIArray)
convert_lonlat.restype = LONLAT_RESTuple
convert_lonlat.errcheck = lonlat_void_array_to_tuple_list

# cleanup
drop_bng_array = lib.drop_int_array
drop_bng_array.argtypes = (BNG_FFIArray, BNG_FFIArray)
drop_bng_array.restype = None
drop_ll_array = lib.drop_float_array
drop_ll_array.argtypes = (LONLAT_FFIArray, LONLAT_FFIArray)
drop_ll_array.restype = None

In [7]:
# def convertbng(lons, lats):
#     """ Single-threaded wrapper """
#     return convert_vec(lons, lats)

def convertbng_threaded(lons, lats):
    """ Multi-threaded lon lat to BNG wrapper """
    return convert_bng(lons, lats)

def convertlonlat_threaded(eastings, northings):
    """ Multi-threaded BNG to lon, lat wrapper """
    return convert_lonlat(eastings, northings)

## Simple test of average conversion speed, Python version

## Test: 1MM random points within the UK

In [8]:
# UK bounding box
N = 55.811741
E = 1.768960
S = 49.871159
W = -6.379880

bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

num_coords = 1000000
lon_ls = list(np.random.uniform(W, E, [num_coords]))
lat_ls = list(np.random.uniform(S, N, [num_coords]))

### Pure Python

In [8]:
%%timeit -r50
[bng(lat, lon) for lat, lon in zip(lat_ls, lon_ls)]

1 loops, best of 10: 547 ms per loop


### Pyproj

In [10]:
%%timeit -r50
zip(*pyproj.transform(wgs84, bng, lon_ls, lat_ls))

1 loop, best of 50: 537 ms per loop


### Multithreaded Rust

In [9]:
%%timeit -r50
zip(*convertbng_threaded(lon_ls, lat_ls))

1 loop, best of 50: 633 ms per loop


## Pyproj is now only 1.2x (~20%) faster than Multithreaded Rust, which is 8–9x faster than pure Python

## Ignore this for now

In [16]:
%%timeit
convertbng_threaded(
    np.random.uniform(W, E, [1000000]),
    np.random.uniform(S, N, [1000000])
)

1 loops, best of 3: 1.95 s per loop


In [19]:
df = pd.DataFrame({
        'lons': lon_ls,
        'lats': lat_ls
        })

In [20]:
%%timeit
df['eastings'], df['northings'] = convertbng_threaded(df.lons.values, df.lats.values))
df.head()

1 loops, best of 3: 2.51 s per loop


In [ ]:
%%timeit
df['eastings'], df['northings'] = df.apply(lambda x: zip(*convert_bng(df.lons, df.lats)), axis=1)